In [1]:
import sys
import cmath
import math
import os

if sys.platform=="cygwin":
    from cyglibra_core import *
elif sys.platform=="linux" or sys.platform=="linux2":
    from liblibra_core import *

from libra_py import models_LVC, units, fgr_py



/home/alexey/miniconda2/envs/libra/lib/python2.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: to-Python converter for std::vector<std::complex<double>, std::allocator<std::complex<double> > > already registered; second conversion method ignored.
  if __name__ == '__main__':
/home/alexey/miniconda2/envs/libra/lib/python2.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: to-Python converter for std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator<int> > > > already registered; second conversion method ignored.
  if __name__ == '__main__':
/home/alexey/miniconda2/envs/libra/lib/python2.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: to-Python converter for boost::python::detail::container_element<std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator<int> > > >, unsigned long, boost::python::detail::final_vector_derived_policies<std::vector<std::vector<int, std::allocator<in

In [3]:
params = models_LVC.get_LVC_set1b()  # parameters are the same as in Xiang's code
 
T = 100.0 # K
beta = 1.0 / (units.kB * T)
tmax = 10 * units.fs2au
dt = 0.1 * units.fs2au
dtau = dt/5.0
dyn_type = 1         # 0 - Condon, 1 - non-Condon

gamma = 1.0   # DA_coupling
s = -1.0     # non-equilibrium parameter

method = 0

In [4]:

ndof = len(params["omega"])
print "ndof = ", ndof
omega = Py2Cpp_double(params["omega"])
coeff = Py2Cpp_double(params["coup"])
d1 = Py2Cpp_double(params["d1"])
d2 = Py2Cpp_double(params["d2"])



ndof =  30


In [15]:
U = MATRIX(ndof, ndof)
omega_nm = normal_modes(omega, coeff, U)
#print "U = "; U.show_matrix()
print "omega_nm = ", Cpp2Py(omega_nm)

omega_nm =  [2.9303174362606456e-05, 0.001730266792188564, 0.0024189999999999945, 0.002863400000000002, 0.003354600000000264, 0.0034833000000000004, 0.003685099999999995, 0.0038834999999999994, 0.004184199999999997, 0.004382799999999994, 0.004461699999999996, 0.004600199999799148, 0.004813735251331605, 0.005131000000000585, 0.005715626072590596, 0.00575669999958431, 0.005947596551398491, 0.007125172717088623, 0.007212900002625722, 0.007602500000030763, 0.007926999999999258, 0.008244931676685226, 0.008579699999993744, 0.016668999999999976, 0.017017966186652432, 0.017045999999992262, 0.01713219043917351, 0.017175999999930792, 0.017396994021224202, 0.19443523097686655]


In [6]:
dE = LVC2GOA_dE(params["Delta1"], params["Delta2"], omega_nm, d1, d2)
print "dE = ", dE
print "Omega_DA = ", params["omega_DA"]
dE = params["omega_DA"]

dE =  -0.132149447238
Omega_DA =  0.0989


In [7]:
#req_nm = compute_req(omega, coeff, y0, U)
req_nm = LVC2GOA_req(omega_nm, d1, d2)
print "req_nm = ", Cpp2Py(req_nm)

req_nm =  [0.0, 0.0001341762612223532, 0.0, 0.0, -33.412299036701455, 0.0, 0.0, 0.0, -0.0011082088906290738, 0.0, 0.0, 0.0, -20.919601903856712, -0.00010784679745755164, -14.249558136238047, -0.004394742920197274, 16.039789901598397, 0.0010440811130974811, -0.00019109126312166744, -10.596559288669988, -7.708793605789494, 24.332550836666428, -0.0008654790971657404, -8.742671880517111, 1.6381274067315381, 0.00021229247528660913, -0.33206143821743045, 0.0003254207677944243, 0.27788699533924116, -4.061890230540401e-07]


In [8]:
print "Omega = ", params["omega"][0]
print "Er(from params) = ", params["Er"]
print "Er(computed) = ", reorganization_energy(omega_nm, req_nm)
y0 = eq_shift(params["Er"], params["omega"][0])
print "y0 = ", y0

Omega =  0.00098662
Er(from params) =  0.0887
Er(computed) =  0.0554905527617
y0 =  213.45036274


In [9]:
#gamma_nm = compute_TT_scaled(U, gamma)
gamma_nm = Py2Cpp_double(params["coup"])
print "gamma_nm = ", Cpp2Py(gamma_nm)

gamma_nm =  [0.0, -9.2519e-06, 0.0, 0.0, -2.0893e-10, 0.0, 0.0, 0.0, 0.00043069, 0.0, 0.0, 0.0, -1.0211e-08, 0.00037411, -2.1549e-09, 4.573e-05, 9.7768e-09, 0.00051583, 0.0009486, 3.1798e-08, 1.2354e-08, -2.0903e-09, -0.00035004, -3.9514e-09, 2.0579e-10, 7.0815e-06, -3.3803e-09, -5.233e-05, 1.0461e-08, 3.0324e-06]


In [10]:
#shift_NE = compute_TT_scaled(U, s)
shift_NE = doubleList()
sz = len(req_nm)
for i in xrange(sz):
    shift_NE.append(s * req_nm[i])

print "shift_NE = ", Cpp2Py(shift_NE)


shift_NE =  [-0.0, -0.0001341762612223532, -0.0, -0.0, 33.412299036701455, -0.0, -0.0, -0.0, 0.0011082088906290738, -0.0, -0.0, -0.0, 20.919601903856712, 0.00010784679745755164, 14.249558136238047, 0.004394742920197274, -16.039789901598397, -0.0010440811130974811, 0.00019109126312166744, 10.596559288669988, 7.708793605789494, -24.332550836666428, 0.0008654790971657404, 8.742671880517111, -1.6381274067315381, -0.00021229247528660913, 0.33206143821743045, -0.0003254207677944243, -0.27788699533924116, 4.061890230540401e-07]


In [12]:
#V = coupling_Condon(gamma, dE, params["Er"], y0)
#print "V = ", V
V = gamma
print V

1.0


In [ ]:
fgr_py.run_NEFGRL_populations(dE, V, omega_nm, gamma_nm,req_nm, shift_NE, method, beta, dyn_type, dtau, tmax, dt, "_res.txt")
